In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('D:/project/homecredit/previous_application.csv')

In [2]:
df_test = df
issue_data = ['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
for name in issue_data:    
#     rename = name + "_ANOM"
#     data[rename] = data[name] == 365243
    df_test[name].replace({365243: np.nan}, inplace = True)
    
# SELLERPLACE_AREA 分行資料有接近一半是-1值，而其他皆為正整數，懷疑是未知
df_test['SELLERPLACE_AREA'].replace({-1: np.nan}, inplace = True)


# Prev將缺失值大於60的欄位去除
miss_value_percent = df_test.isnull().sum().sort_values(ascending=False)/len(df_test)
miss_value_percent = (miss_value_percent * 100).round(decimals=2)

if (miss_value_percent>60).any():
    delect_colomns = list(miss_value_percent[miss_value_percent>60].index)
    for n in delect_colomns:
        df_test = df_test.drop(columns=n)
        
# AMT_CREDIT空值只有一個直接補0
df_test['AMT_CREDIT'].replace({np.nan: 0}, inplace = True)

print(f'刪了欄位 {delect_colomns} ')

刪了欄位 ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_FIRST_DRAWING'] 


In [3]:
test = df_test.copy()

test['ANNUITY/CREDIT'] = test['AMT_ANNUITY'] / test['AMT_CREDIT']
test['DOWN_PAYMENT/ANNUITY'] = test['AMT_DOWN_PAYMENT'] / test['AMT_ANNUITY']
test['DOWN_PAYMENT/CREDIT'] = test['AMT_DOWN_PAYMENT'] / test['AMT_CREDIT']
test['DOWN_PAYMENT/ANNUITY'] = test['AMT_DOWN_PAYMENT'] / test['AMT_ANNUITY']
test['GOODS_PRICE/CREDIT'] = test['AMT_GOODS_PRICE'] / test['AMT_CREDIT']
test['APPLICATION/CREDIT'] = test['AMT_APPLICATION'] / test['AMT_CREDIT']
test['APPLICATION/GOODS_PRICE'] = test['AMT_APPLICATION'] / test['AMT_GOODS_PRICE']
test['DAYS_LAST_DUE-DAYS_TERMINATION'] = test['DAYS_LAST_DUE'] - test['DAYS_TERMINATION']

# 將有特殊狀態的欄位新增 (onehot)

onehot_list = ['NAME_CONTRACT_STATUS',
'NAME_TYPE_SUITE',
'CODE_REJECT_REASON',
'NAME_PAYMENT_TYPE',
'NAME_PRODUCT_TYPE',
'NFLAG_INSURED_ON_APPROVAL',
'PRODUCT_COMBINATION',
'NAME_SELLER_INDUSTRY',
'NAME_YIELD_GROUP']

test = pd.get_dummies(test, columns = onehot_list , dummy_na=True)

# 針對previous_application 新增的欄位做計算

test1=test.copy()

# 把float64拿出來作為欄位要處理的鍵值對  ex: 'AMT_ANNUITY': ['sum', 'max', 'min', 'mean'] ,  ......'
new_feature = [
              'AMT_ANNUITY',
              'AMT_APPLICATION',
              'AMT_CREDIT',
              'AMT_DOWN_PAYMENT',
              'AMT_GOODS_PRICE',
              'DAYS_LAST_DUE',
              'DAYS_TERMINATION',
              'ANNUITY/CREDIT',
              'DOWN_PAYMENT/ANNUITY',
              'DOWN_PAYMENT/CREDIT',
              'DOWN_PAYMENT/ANNUITY',
              'GOODS_PRICE/CREDIT',
              'APPLICATION/CREDIT',
              'APERMINAPLICATION/GOODS_PRICE',
              'DAYS_LAST_DUE-DAYS_TTION']

dict_type = {}
for i in new_feature:
        dict_type[i] = ['max','min']
num_aggregations = dict_type

test_1 = test.groupby(['SK_ID_CURR']).agg({**num_aggregations})

# 重新排列欄位
columns = []

for m in test_1.columns.levels[0]:
    for n in test_1.columns.levels[1]:
#         if m == 'SK_ID_PREV':
#             columns.append('count')
#             break
        columns.append(f'PREV_{m}_{n}')
        
test_1.columns = columns

test_1.reset_index(level=('SK_ID_CURR'),inplace=True)

# *****************************************************************************************************************

# 針對previous_application類別型態onehot後做計算

test2 = test.copy()

# 把onehot出來欄位變成要處理的鍵值對  ex:'NAME_CONTRACT_TYPE_Cash loans': ['count'] ,  ......'
dict_type = {}
for i in list(test.columns[91:]):
        dict_type[i] = ['sum']
num_aggregations = dict_type
# 計算出現的次數
count = {'SK_ID_PREV':'count'}

test_2 = test2.groupby(['SK_ID_CURR']).agg({**count,**num_aggregations})


# 重新排列欄位
columns = []

for m in test_2.columns.levels[0]:
    for n in test_2.columns.levels[1][1:]:
        if m == 'SK_ID_PREV':
            columns.append('count')
            break
        columns.append(f'PREV_{m}_rate')
        
test_2.columns = columns

test_2.reset_index(level=('SK_ID_CURR'),inplace=True)

# 計算比例
ID_count_t = test_2['count'].values.reshape(338857,1)
test_rate = (test_2.iloc[:,2:]/ID_count_t)

# 組在一起
test_2_rate = pd.concat([test_2['SK_ID_CURR'],test_rate], axis = 1)

# *****************************************************************************************************************

# SELLERPLACE_AREA集中程度(在同一家的最大出現次數除以此人的貸款數)

test4 = test.copy()
test4['Total_count'] = test4['SK_ID_PREV']
# 先以各ID及各分行去計次
test_4 = test4.groupby(['SK_ID_CURR','SELLERPLACE_AREA']).agg({'Total_count':'count'})
test_4.reset_index(level=('SK_ID_CURR','SELLERPLACE_AREA'),inplace=True)
# 新增一欄作為各ID在哪間分行使用作多的欄位
test_4['count_max'] =test_4['Total_count']
test_4 = test_4.groupby(['SK_ID_CURR']).agg({'count_max':'max','Total_count':'sum'})
# 新增一欄計算比例
test_4['SELLERPLACE_AREA_HIGH_rate'] = test_4['count_max']/test_4['Total_count']
test_4.reset_index(level=('SK_ID_CURR'),inplace=True)
# 只留比例
test_4 = test_4.drop(columns=['count_max','Total_count'],axis=1)
                     

In [4]:
file_list = [test_1,test_2_rate,test_4]

# 建一個空的df
empty_df = pd.DataFrame(columns=['SK_ID_CURR'])

for i in file_list:    
    empty_df = pd.merge(empty_df, i,how='outer')

# empty_df.to_csv('D:/project/homecredit/final/previous_application_ETL_v2.csv')

合併到最終表